### קוד מבוא

In [26]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [27]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [28]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [29]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [30]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [31]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [32]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [33]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [34]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


### קוד

#### מידע של מלל

In [35]:
gov=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\שיתופי פעולה\תחזיות כלל ארציות מועצה הלאומית לכלכלה\210512_תוצאות_סופיות','טיוטת תוצאות לתרחישי אוכלוסייה אזוריים - 2020-2050 - טיוטה')

In [36]:
gov_bau=gov.query('proj=="medium" & housing_adjusted ==False & year==2050 & (district=="Jerusalem District" | district=="Yehuda and Shomron")')

In [37]:
gov_plan=gov.query('proj=="medium" & housing_adjusted ==True & year==2050 & (district=="Jerusalem District" | district=="Yehuda and Shomron")')

In [38]:
gov_bau_pivot=gov_bau.pivot_table(values='pop',index=['district','group'],aggfunc=sum)

In [39]:
gov_plan_pivot=gov_plan.pivot_table(values='pop',index=['district','group'],aggfunc=sum)

In [40]:
gov_bau_pivot=gov_bau_pivot.loc[[('Jerusalem District',           'arab'),
            ('Jerusalem District',         'haredi'),
            ('Jerusalem District', 'jew_not_haredi'),
            
            ('Yehuda and Shomron',         'haredi'),
            ('Yehuda and Shomron', 'jew_not_haredi')]]

In [41]:
gov_plan_pivot=gov_plan_pivot.loc[[('Jerusalem District',           'arab'),
            ('Jerusalem District',         'haredi'),
            ('Jerusalem District', 'jew_not_haredi'),
            
            ('Yehuda and Shomron',         'haredi'),
            ('Yehuda and Shomron', 'jew_not_haredi')]]

#### מידע של צתאל

In [42]:
jtmt=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\join_forecast','230720_forecast_2020_till_2050_jtmt')

In [43]:
jtmt.loc[jtmt['main_secto']=="U_Orthodox",'group']='haredi'

jtmt.loc[jtmt['main_secto']=="arabs_behined_seperation_wall",'group']='arab'

jtmt.loc[jtmt['main_secto']=="Arab",'group']='arab'

jtmt.loc[jtmt['main_secto']=="Jewish",'group']='jew_not_haredi'

jtmt.loc[jtmt['zonetype']=="Jerusalem",'district']='Jerusalem District'

jtmt.loc[jtmt['zonetype']=="Judea and Samaria",'district']='Yehuda and Shomron'

In [44]:
jtmt_pivot=jtmt.pivot_table(values='pop_without_dorms_yeshiva_2050',index=['district','group'],aggfunc=sum)

#### תרחיש דיור לפי צתאל

In [45]:
iplan=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\iplan\join_forecast','230720_forecast_2020_till_2050_iplan')

In [46]:
iplan.loc[iplan['main_secto']=="U_Orthodox",'group']='haredi'

iplan.loc[iplan['main_secto']=="arabs_behined_seperation_wall",'group']='arab'

iplan.loc[iplan['main_secto']=="Arab",'group']='arab'

iplan.loc[iplan['main_secto']=="Jewish",'group']='jew_not_haredi'

iplan.loc[iplan['zonetype']=="Jerusalem",'district']='Jerusalem District'

iplan.loc[iplan['zonetype']=="Judea and Samaria",'district']='Yehuda and Shomron'

In [47]:
iplan_pivot=iplan.pivot_table(values='pop_without_dorms_yeshiva_2050',index=['district','group'],aggfunc=sum)

#### תרחיש עסקים כרגיל לפי צתאל

In [48]:
bau=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\bau\join_forecast','230720_forecast_2020_till_2050_bau')

In [49]:
bau.loc[bau['main_secto']=="U_Orthodox",'group']='haredi'

bau.loc[bau['main_secto']=="arabs_behined_seperation_wall",'group']='arab'

bau.loc[bau['main_secto']=="Arab",'group']='arab'

bau.loc[bau['main_secto']=="Jewish",'group']='jew_not_haredi'

bau.loc[bau['zonetype']=="Jerusalem",'district']='Jerusalem District'

bau.loc[bau['zonetype']=="Judea and Samaria",'district']='Yehuda and Shomron'

In [50]:
bau_pivot=bau.pivot_table(values='pop_without_dorms_yeshiva_2050',index=['district','group'],aggfunc=sum)

#### חיבור טבלאות

In [52]:
jtmt_pivot.join(gov_plan_pivot).join(gov_bau_pivot,lsuffix='_iplan',rsuffix='_bau').join(iplan_pivot,rsuffix='_iplan_from_jtmt').join(bau_pivot,rsuffix='_bau_from_jtmt').to_excel('delet.xlsx')